# Capacities analysis

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

Aca queremos entender cuantas personas trabajan en las ips mas importantes/relevantes a la hora de combatir la pandemia. Para esto, usaremos como proxy la capacidad en camas/pacientes de cada ips junto con un estimado de personas necesarias por cama/paciente

In [12]:
capacities = pd.read_parquet('../data/02_processed/capacities_important.parquet')

In [13]:
capacities.head()

,nom_sede_ips,nit_ips,naturaleza,nivel_atencion,grupo_capacidad,descripcion_capacidad,capacidad
1,CAMPONUEVO CISNE 2,830100595,Privada,NaN,CAMAS,Psiquiatría,15.0
6,PMA MARLY,830014121,Privada,NaN,SALAS,Quirófano,1.0
8,FUNDACION HOSPITAL DE LA MISERICORDIA,899999123,Privada,NaN,CAMAS,Pediátrica,310.0
10,HOSPITAL UNIVERSITARIO MAYOR-MEDERI,900210981,Privada,NaN,CAMAS,Adultos,452.0
13,HOSPITAL MILITAR CENTRAL,830040256,Pública,3.0,CAMAS,Adultos,294.0


In [11]:
capacities.describe(include='all')

,nom_sede_ips,nit_ips,naturaleza,nivel_atencion,grupo_capacidad,descripcion_capacidad,capacidad
count,1388,1388,1388,239.000000,1388,1388,1388.000000
unique,608,252,2,NaN,2,22,NaN
top,UNIDAD DE SERVICIOS DE SALUD SIMÓN BOLÍVAR,800215908,Privada,NaN,CAMAS,Procedimientos,NaN
freq,16,203,1149,NaN,746,358,NaN
mean,NaN,NaN,NaN,2.958159,NaN,NaN,14.355187
std,NaN,NaN,NaN,0.238884,NaN,NaN,32.544661
min,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000
25%,NaN,NaN,NaN,3.000000,NaN,NaN,1.000000
50%,NaN,NaN,NaN,3.000000,NaN,NaN,4.000000
75%,NaN,NaN,NaN,3.000000,NaN,NaN,12.000000


In [9]:
capacities.nit_ips.nunique()

252

In [7]:
# es importante saber cuales son las camas que nos interesan, tal vez psiquatria no
capacities.groupby(['grupo_capacidad', 'descripcion_capacidad']).nit_ips.count()

grupo_capacidad  descripcion_capacidad                      
CAMAS            Adultos                                        141
                 Cuidado Agudo Mental                             8
                 Cuidado Intensivo Adulto                        83
                 Cuidado Intensivo Neonatal                      58
                 Cuidado Intensivo Pediátrico                    29
                 Cuidado Intermedio Adulto                       76
                 Cuidado Intermedio Mental                        2
                 Cuidado Intermedio Neonatal                     58
                 Cuidado Intermedio Pediátrico                   23
                 Cuidado básico neonatal                         50
                 Farmacodependencia                              10
                 Institución Paciente Crónico                    28
                 Obstetricia                                     65
                 Pediátrica                            

Importantes:

- Adultos (Aunque tal vez un poco menos)
- Todas las que mencionan cuidado

Ademas, solo nos quedamos con la categoria `CAMAS`

In [18]:
capacities = (
    capacities.
    assign(capacidad_clave = lambda x: (x.descripcion_capacidad=='Adultos') | x.descripcion_capacidad.str.startswith('Cuidadp')))

In [19]:
capacities_filtered = capacities.loc[lambda x: x.capacidad_clave]

El max por nit y categoria

In [27]:
capacities_per_nit = (capacities_filtered.
     groupby(['nit_ips', 'descripcion_capacidad']).
     agg({'capacidad': 'max'}).
     reset_index().
     groupby('nit_ips').
     agg({'capacidad':'sum'}).
     reset_index()
)

In [29]:
capacities_per_nit

,nit_ips,capacidad
0,800006602,3.0
1,800074904,5.0
2,800085486,155.0
3,800117564,12.0
4,800118223,20.0
...,...,...
65,900702981,67.0
66,900958564,108.0
67,900959048,158.0
68,900959051,139.0


In [30]:
capacities_per_nit.to_parquet('../data/02_processed/capacities_per_nit.parquet')